In [ ]:
# Run with python 2.7
import numpy as np
import shutil
import os
import pandas as pd
import numpy as np
import FATS
import glob
import numpy as np

# Root directory of the project
ROOT_DIR = os.path.abspath("../code")
# Import code
sys.path.append(ROOT_DIR)

from preprocessing import sigma_clipping


In [ ]:
'--------------------------------------------------------------------------------------------------------------------------'
# Feature extraction
# select_features = ['Skew','Mean', 'Std', 'SmallKurtosis', 'Rcs', 'Amplitude', 'FluxPercentileRatioMid20', 'FluxPercentileRatioMid35', \
#'FluxPercentileRatioMid50', 'FluxPercentileRatioMid65', 'FluxPercentileRatioMid80', 'Freq1_harmonics_amplitude_0', 'Meanvariance', \
#'PeriodLS', 'Period_fit', 'PercentDifferenceFluxPercentile']

# data_columns : ['magnitude', 'time','error']
# data_dir     : The directory where the data is stored
# type_ID_list : List of file name for a specific class/category
# save_dir     : The directory you want to save your features


'--------------------------------------------------------------------------------------------------------------------------'

def feature_extraction(data_dir, true_class, data_columns, select_features, save_dir):
    files  = glob.glob(data_dir+'Type'+str(true_class)+'/*.dat')
    #files  = files[0:3]
    print(len(files))
    feature_file = pd.DataFrame()
    for file in files:
        filename = os.path.basename(str(file))
        file_name = filename[0:-4]
        print(file_name)

        data          = np.loadtxt(file)
        date          = data[:,0]
        mag     = data[:,1]
        mag_err = data[:,2]

        time, magnitude, magnitude_err = sigma_clipping(date,mag,mag_err,threshold=3,iteration=2)
        print('length of data is {} before clipping and length of data after clipping is {}'.format(len(mag),len(magnitude)))

        # Feature Extraction Method
        lc              = np.array([ magnitude, time, magnitude_err])
        feature_extract = FATS.FeatureSpace(Data=data_columns, featureList = select_features)
        features_cal    = feature_extract.calculateFeature(lc)
        features_name   = features_cal.result(method='features')
        features_value  = features_cal.result(method='array')
        features_df     = pd.DataFrame(features_value.reshape(1,len(select_features)))
        features_df['File_Name'] = file_name
        features_df['True_class_labels'] = true_class
        feature_file = feature_file.append(features_df)

    feature_file.to_csv(save_dir+'Type'+str(true_class)+'_features.csv',index=None)
  

In [ ]:
# Here we only need to change foldername and list_class(classes(13)) - 13 can be changed from 0 to 13 class 		
# Creating a directory and overwrite directory if it exists

features = ['Skew', 'Mean', 'Std', 'SmallKurtosis','Amplitude', 'Meanvariance']
# Call function to extract features 
feature_extraction(data_dir='../data/training_set/', true_class=12, data_columns = ['magnitude', 'time','error'], \
    select_features = features, save_dir = "../data/training_set_features/")
